# ADVANCED PSE+ STAKEHOLDER SUMMIT I 2023: 
# Interactive Code Demonstration Using WaterTAP

### Today's demonstration will show 
- Part 1: how to build, initialize, simulate, and optimize a flowsheet for multiperiod analysis. The demonstration will demonstrate the use of a reverse osmosis (RO) unit model.
- Part 2: :
    - Public Github Repository: https://github.com/watertap-org/watertap
    - Documentation: https://watertap.readthedocs.io/en/stable/
    - Installer for User Interface: https://watertap-org.github.io/

# Overall approach to multiperiod flowsheets
<p align="center">
  <img src="assets/MP_Framework.png" height="320">
</p>

## Part 1: Build, setup, and simulate the multiperiod RO+PV+Battery flowsheet


<p align="center">
  <img src="assets/RO_PV_Batt.svg">
</p>

## Multiperiod Setup
#### Import Pyomo, IDAES, and WaterTAP packages

In [1]:
# Pyomo imports
from pyomo.environ import ConcreteModel, Objective, Var, value, units as pyunits

# IDAES imports
from idaes.core import FlowsheetBlock

from idaes.apps.grid_integration.multiperiod.multiperiod import MultiPeriodModel
from idaes.core.solvers.get_solver import get_solver


#### Build Model

In [2]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)

#### Import and define the Reverse Osmosis unit model

In [3]:
from steady_state_flowsheets.simple_RO_unit import ROUnit
m.fs.RO = ROUnit()

#### Import and define the Battery model

In [4]:
from steady_state_flowsheets.battery import BatteryStorage
m.fs.battery = BatteryStorage()
m.fs.battery.initialize()


2023-10-05 17:32:53 [INFO] idaes.init.fs.battery: Battery initialization status optimal - Optimal Solution Found.


#### Load PV surrogate

In [5]:
from idaes.core.surrogate.pysmo_surrogate import PysmoSurrogate
PV_surrogate = PysmoSurrogate.load_from_file('assets/demo_surrogate.json')

2023-10-05 17:32:53 [INFO] idaes.core.surrogate.pysmo_surrogate: Decode surrogate. type=rbf
Default parameter estimation method is used.

Parameter estimation method:  algebraic
Gaussian basis function is used.
Basis function:  gaussian
Regularization done:  True


### Define critical variables and add energy balance constraints

In [6]:
from steady_state_flowsheets.system import *
define_system_vars(m)
add_steady_state_constraints(m)

#### Add Table Here

In [7]:
solver = get_solver()
results = solver.solve(m)

In [8]:
for v in m.fs.component_data_objects(ctype=Var, active=True, descend_into=True):
        print(f'{str(v):<40s}', f'{value(v):<10,.1f}', pyunits.get_units(v))

fs.pv_to_ro                              194.4      kW
fs.grid_to_ro                            374.8      kW
fs.curtailment                           402.4      kW
fs.elec_price                            0.1        USD_2021
fs.elec_generation                       1,000.0    kW
fs.pv_gen                                1,000.0    kW
fs.ro_elec_req                           1,000.0    kW
fs.electricity_price                     0.1        USD_2021
fs.battery.nameplate_power               150,189.1  kW
fs.battery.nameplate_energy              278,058.9  kWh
fs.battery.initial_state_of_charge       121,922.4  kWh
fs.battery.initial_energy_throughput     99,811.5   kWh
fs.battery.elec_in[0.0]                  403.2      kWh
fs.battery.elec_out[0.0]                 375.0      kWh
fs.battery.state_of_charge[0.0]          121,910.7  kWh
fs.battery.energy_throughput[0.0]        100,200.6  kWh


### Create a Pyomo concrete model and steady-state flowsheet

## Refresh - approach to multiperiod flowsheets
<p align="center">
  <img src="assets/MP_Framework_2.png" height="380">
</p>

In [9]:
def get_pv_ro_variable_pairs(t1, t2):
    """
    This function returns pairs of variables that need to be connected across two time periods

    Args:
        t1: current time block
        t2: next time block

    Returns:
        None
    """
    return [
        (t1.fs.battery.state_of_charge[0], t2.fs.battery.initial_state_of_charge),
        (t1.fs.battery.energy_throughput[0], t2.fs.battery.initial_energy_throughput),
        (t1.fs.battery.nameplate_power, t2.fs.battery.nameplate_power),
        (t1.fs.battery.nameplate_energy, t2.fs.battery.nameplate_energy),
        ]

#### Build the steady-state flowsheet as a function

In [10]:
def build_pv_battery_flowsheet(
        m = None,
        pv_gen = 1000,
        electricity_price = 0.1,
        pv_oversize = 1):
    """

    Returns:
        object: A Pyomo concrete optimization model and flowsheet
    """
    
    build_system()

    return m

In [11]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
build_pv_battery_flowsheet(m)

NameError: name 'build_system' is not defined

In [ ]:
results = solver.solve(m)

ValueError: No variables appear in the Pyomo model constraints or objective. This is not supported by the NL file interface

In [ ]:
for v in m.fs.component_data_objects(ctype=Var, active=True, descend_into=True):
        print(f'{str(v):<40s}', f'{value(v):<10,.1f}', pyunits.get_units(v))

fs.pv_to_ro                              194.4      kW
fs.grid_to_ro                            374.8      kW
fs.curtailment                           402.4      kW
fs.elec_price                            0.1        USD_2021
fs.elec_generation                       1,000.0    kW
fs.pv_gen                                1,000.0    kW
fs.ro_elec_req                           1,000.0    kW
fs.electricity_price                     0.1        USD_2021
fs.battery.nameplate_power               149,705.9  kW
fs.battery.nameplate_energy              279,467.3  kWh
fs.battery.initial_state_of_charge       118,932.0  kWh
fs.battery.initial_energy_throughput     99,694.1   kWh
fs.battery.elec_in[0.0]                  403.2      kWh
fs.battery.elec_out[0.0]                 375.0      kWh
fs.battery.state_of_charge[0.0]          118,920.3  kWh
fs.battery.energy_throughput[0.0]        100,083.2  kWh


#### Define the flowsheet options

In [ ]:
def load_flowsheet_options(mp,
        n_time_points= 24,
        ro_capacity = 6000, # m3/day
        ro_elec_req = 1000, # kW
        pv_oversize = 1,
        surrogate = None):
    
    flowsheet_options={ t: { 
                            "pv_gen": 1000,
                            "electricity_price": 0.1,
                            "ro_capacity": ro_capacity, 
                            "ro_elec_req": ro_elec_req,
                            "pv_oversize": pv_oversize} 
                            for t in range(n_time_points)
    }

    # create the multiperiod object
    mp.build_multi_period_model(
        model_data_kwargs=flowsheet_options,
        flowsheet_options={ "ro_capacity": ro_capacity, 
                            "ro_elec_req": ro_elec_req},)

In [ ]:
def create_multiperiod_pv_battery_model(
        n_time_points= 24,        
    ):
    
    mp = MultiPeriodModel(
        n_time_points=n_time_points,
        process_model_func=load_flowsheet_options,
        linking_variable_func=get_pv_ro_variable_pairs,
        initialization_func=fix_dof_and_initialize,
        unfix_dof_func=unfix_dof,
    )

    # surrogate = load_surrogate()
    load_flowsheet_options(mp)
    mp.blocks[0].process.fs.battery.initial_state_of_charge.fix(0)
    add_pv_ro_constraints(mp)
    
    return mp

In [ ]:
mp = create_multiperiod_pv_battery_model()
results = solver.solve(mp)

[+   0.00] Beginning the formulation of the multiperiod optimization problem.
2023-10-05 17:04:26 [INFO] idaes.apps.grid_integration.multiperiod.multiperiod: ...Constructing the flowsheet model for blocks[0]


TypeError: load_flowsheet_options() got an unexpected keyword argument 'pv_gen'

In [ ]:
labels = ['pv_size', 'battery_power', 'battery_energy', 'LCOW']
for idx, v in enumerate([mp.blocks[0].process.fs.pv_size, mp.blocks[0].process.fs.battery.nameplate_power, mp.blocks[0].process.fs.battery.nameplate_energy, mp.LCOW]):
    print(f'{labels[idx]:<20s}', f'{value(v):<10,.2f}')

pv_size              1,000.00  
battery_power        27.60     
battery_energy       40.43     
LCOW                 0.41      


In [ ]:
from util.visualize import create_plot
create_plot(mp, elec_prices)

NameError: name 'elec_prices' is not defined